<h3> 참조 사이트 목록 </h3>
<ol>
    <li> Kaggle Environment :
        <a href=https://github.com/Kaggle/kaggle-environments>https://github.com/Kaggle/kaggle-environments</a>
    <li> Stable3 :
        <a href=https://stable-baselines3.readthedocs.io/en/master/>https://stable-baselines3.readthedocs.io/en/master/
        </a>
    <li> OpenAI - gym :
        <a href=https://www.gymlibrary.dev/api/core/#>https://www.gymlibrary.dev/api/core/#
        </a>
    <li> Kaggle Notebook :
        <a href=https://www.kaggle.com/code/basu369victor/my-first-attempt-with-reinforcement-learning>
            https://www.kaggle.com/code/basu369victor/my-first-attempt-with-reinforcement-learning
        </a>

</ol>

In [9]:
from kaggle_environments import make
import kaggle_environments
import numpy as np
import torch
import gym

<h6> ==== 게임규칙 설명용 ====

In [36]:
env = make("connectx")
env.play([None, "random"])

In [6]:
class ConnectX(gym.Env):
    def __init__(self):
        self.env = make("connectx", debug=True)
        self.pair = [None,"negamax"]
        self.config = self.env.configuration
        self.trainer = self.env.train(self.pair)
        
        # Define required gym fields (examples):
        config = self.env.configuration
        self.action_space = gym.spaces.Discrete(config.columns)
        self.observation_space = gym.spaces.Discrete(config.columns * config.rows)
        
    def step(self,action):
        next_state, reward, terminated, info = self.trainer.step(action)
        next_state = np.array(next_state["board"])
        return next_state, reward, terminated, info

    def reset(self):
        return self.trainer.reset()
        return np.array(self.trainer.reset()["board"])

    def render(self, **kwargs):
        return self.env.render(**kwargs)


env = ConnectX()

In [155]:
import torch
from collections import namedtuple
import matplotlib.pyplot as plt

class PolicyNet(torch.nn.Module):
    def __init__(self, input_size, output_size, hidden_layer_size=64):
        super(PolicyNet, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_layer_size)
        self.fc2 = torch.nn.Linear(hidden_layer_size, output_size)
        self.softmax = torch.nn.Softmax(dim=0)

    def forward(self, x):
        x = torch.tensor(x).float()
        return self.softmax(self.fc2(torch.nn.functional.relu(self.fc1(x))))

    def get_action_and_logp(self, x):
        action_prob = self.forward(x)
        m = torch.distributions.Categorical(action_prob)
        action = m.sample()
        logp = m.log_prob(action)
        return action.item(), logp

    def act(self, x):
        action, _ = self.get_action_and_logp(x)
        return action

class ValueNet(torch.nn.Module):
    def __init__(self, input_size, hidden_layer_size=64):
        super(ValueNet, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_layer_size)
        self.fc2 = torch.nn.Linear(hidden_layer_size, 1)

    def forward(self, x):
        x = torch.from_numpy(x).float()
        return self.fc2(torch.nn.functional.relu(self.fc1(x)))

def policyGradient(env, max_num_steps=1000, gamma=0.98, lr=0.01,
                   num_traj=10, num_iter=200):
#     input_size = gym.spaces.Discrete(env.configuration.columns * env.configuration.rows)
#     output_size = gym.spaces.Discrete(env.configuration.columns)
#     input_size = env.configuration.columns * env.configuration.rows
#     output_size = env.configuration.columns
    input_size = env.observation_space.n
    output_size = env.action_space.n
    Trajectory = namedtuple('Trajectory', 'states actions rewards dones logp')

    def collect_trajectory():
        state_list = []
        action_list = []
        reward_list = []
        dones_list = []
        logp_list = []
        state = env.reset()
        done = False
        steps = 0
        while not done and steps <= max_num_steps:
            action, logp = policy.get_action_and_logp(state)
            newstate, reward, done, _ = env.step(action)
            state_list.append(state)
            action_list.append(action)
            reward_list.append(reward)
            dones_list.append(done)
            logp_list.append(logp)
            steps += 1
            state = newstate

        traj = Trajectory(states=state_list, actions=action_list,
                          rewards=reward_list, logp=logp_list, dones=dones_list)
        return traj

    def calc_returns(rewards):
        dis_rewards = [gamma**i * r for i, r in enumerate(rewards)]
        return [sum(dis_rewards[i:]) for i in range(len(dis_rewards))]

    policy = PolicyNet(input_size, output_size)
    policy_optimizer = torch.optim.Adam(policy.parameters(), lr=lr)

    value = ValueNet(input_size)
    value_optimizer = torch.optim.Adam(value.parameters(), lr=lr)

    mean_return_list = []
    for it in range(num_iter):
        traj_list = [collect_trajectory() for _ in range(num_traj)]
        returns = [calc_returns(traj.rewards) for traj in traj_list]

        #====================================#
        # policy gradient with base function #
        #====================================#
        policy_loss_terms = [-1. * traj.logp[j] * (returns[i][j] - value(traj.states[j]))
                             for i, traj in enumerate(traj_list) for j in range(len(traj.actions))]

        #====================================#
        # policy gradient with reward-to-go  #
        #====================================#
        #policy_loss_terms = [-1. * traj.logp[j] * (torch.Tensor([returns[i][j]]))
        #                     for i, traj in enumerate(traj_list) for j in range(len(traj.actions))]

        #====================================#
        # policy gradient                    #
        #====================================#
        #policy_loss_terms = [-1. * traj.logp[j] * (torch.Tensor([returns[i][0]]))
        #                     for i, traj in enumerate(traj_list) for j in range(len(traj.actions))]

        policy_loss = 1. / num_traj * torch.cat(policy_loss_terms).sum()
        policy_optimizer.zero_grad()
        policy_loss.backward()
        policy_optimizer.step()

        value_loss_terms = [1. / len(traj.actions) * (value(traj.states[j]) - returns[i][j])**2.
                            for i, traj in enumerate(traj_list) for j in range(len(traj.actions))]
        value_loss = 1. / num_traj * torch.cat(value_loss_terms).sum()
        value_optimizer.zero_grad()
        value_loss.backward()
        value_optimizer.step()

        mean_return = 1. / num_traj * \
            sum([traj_returns[0] for traj_returns in returns])
        mean_return_list.append(mean_return)
        if it % 10 == 0:
            print('Iteration {}: Mean Return = {}'.format(it, mean_return))

    return policy, mean_return_list

env._max_episode_steps=500
agent, mean_return_list = policyGradient(env, num_iter=500, max_num_steps=500, gamma=1.0, num_traj=5)

plt.plot(mean_return_list)
plt.xlabel('Iteration')
plt.ylabel('Mean Return')
# plt.savefig('pg1_returns.png', format='png', dpi=300)

RuntimeError: Could not infer dtype of Struct

In [49]:
# libraries
import gym
import collections
import random

# pytorch library is used for deep learning
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# hyperparameters
learning_rate = 0.0005
gamma = 0.98
buffer_limit = 50000        # size of replay buffer
batch_size = 32

class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit)    # double-ended queue
    
    def put(self, transition):
        self.buffer.append(transition)

    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []

        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])

        return torch.tensor(s_lst, dtype=torch.float), torch.tensor(a_lst), \
               torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch.float), \
               torch.tensor(done_mask_lst)

    def size(self):
        return len(self.buffer)

class DuelingQnet(nn.Module):
    def __init__(self):
        super(DuelingQnet, self).__init__()
        self.fc1 = nn.Linear(43, 128)
        self.fc_value = nn.Linear(128, 128)
        self.fc_adv = nn.Linear(128, 128)
        self.value = nn.Linear(128, 1)
        self.adv = nn.Linear(128, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        v = F.relu(self.fc_value(x))
        a = F.relu(self.fc_adv(x))
        v = self.value(v)
        a = self.adv(a)
        a_avg = torch.mean(a)
        q = v + a - a_avg
        return q

    def sample_action(self, obs, epsilon):
        out = self.forward(obs)
        coin = random.random()
        if coin < epsilon:
            return random.randint(0,1)
        else : 
            return out.argmax().item()  

class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()
        self.fc1 = nn.Linear(43, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
      
    def sample_action(self, obs, epsilon):
        out = self.forward(obs)
        coin = random.random()
        if coin < epsilon:
            return random.randint(0,1)
        else : 
            return out.argmax().item()   

def train(q, q_target, memory, optimizer):
    for i in range(10):
        s,a,r,s_prime,done_mask = memory.sample(batch_size)

        q_out = q(s)
        q_a = q_out.gather(1,a)

        # DQN
        #max_q_prime = q_target(s_prime).max(1)[0].unsqueeze(1)

        # Double DQN
        argmax_Q = q(s_prime).max(1)[1].unsqueeze(1)
        max_q_prime = q_target(s_prime).gather(1, argmax_Q)

        target = r + gamma * max_q_prime * done_mask
        
        # MSE Loss
        loss = F.mse_loss(q_a, target)

        # Smooth L1 Loss
        #loss = F.smooth_l1_loss(q_a, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
def preprocess(state):
    result = state.board[:]
    result.append(state.mark)
    return result

#     env = gym.make('CartPole-v1')
env = ConnectX()
q = DuelingQnet()
q_target = DuelingQnet()
q_target.load_state_dict(q.state_dict())
memory = ReplayBuffer()

print_interval = 20
score = 0.0
optimizer = optim.Adam(q.parameters(), lr=learning_rate)

for n_epi in range(1000):
    epsilon = max(0.01, 0.08 - 0.01*(n_epi/200)) #Linear annealing from 8% to 1%
    s = env.reset()
    done = False

    while not done:
        stat = np.array(preprocess(s))
        a = q.sample_action(torch.from_numpy(stat).float(), epsilon)
        s_prime, r, done, info = env.step(a)
        done_mask = 0.0 if done else 1.0
        if done:
            r = a
        memory.put((s,a,r/100,s_prime, done_mask))
        stat = s_prime

        score += r
        if done:
            break

    if memory.size()>2000:
        train(q, q_target, memory, optimizer)

    if n_epi%print_interval==0 and n_epi!=0:
        q_target.load_state_dict(q.state_dict())
        print("n_episode :{}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(
                                                        n_epi, score/print_interval, memory.size(), epsilon*100))
        score = 0.0

env.close()

Invalid Action: Invalid column: 0
Invalid Action: Invalid column: 0
Invalid Action: Invalid column: 0
Invalid Action: Invalid column: 0
Invalid Action: Invalid column: 0
Invalid Action: Invalid column: 0
Invalid Action: Invalid column: 0
Invalid Action: Invalid column: 0
Invalid Action: Invalid column: 0
Invalid Action: Invalid column: 0
Invalid Action: Invalid column: 0
Invalid Action: Invalid column: 0
Invalid Action: Invalid column: 0
n_episode :20, score : 0.0, n_buffer : 117, eps : 7.9%
Invalid Action: Invalid column: 0
Invalid Action: Invalid column: 0
Invalid Action: Invalid column: 0
Invalid Action: Invalid column: 0
Invalid Action: Invalid column: 0
Invalid Action: Invalid column: 0
Invalid Action: Invalid column: 0
Invalid Action: Invalid column: 0
Invalid Action: Invalid column: 0
Invalid Action: Invalid column: 0
Invalid Action: Invalid column: 0
Invalid Action: Invalid column: 0
Invalid Action: Invalid column: 0
Invalid Action: Invalid column: 0
Invalid Action: Invalid col

Invalid Action: Invalid column: 0
Invalid Action: Invalid column: 0
Invalid Action: Invalid column: 0
Invalid Action: Invalid column: 0
Invalid Action: Invalid column: 0
Invalid Action: Invalid column: 0


TypeError: must be real number, not Struct